In [9]:
! apt-get install default-jre
! java -version
! pip install h2o

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.8-59ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
openjdk version "1.8.0_171"
OpenJDK Runtime Environment (build 1.8.0_171-8u171-b11-0ubuntu0.17.10.1-b11)
OpenJDK 64-Bit Server VM (build 25.171-b11, mixed mode)


In [37]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
h2o.init()

from google.colab import files

from h2o.estimators import H2ORandomForestEstimator, H2OGradientBoostingEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator



Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,4 hours 43 mins
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.8
H2O cluster version age:,2 days
H2O cluster name:,H2O_from_python_unknownUser_d1p26n
H2O cluster total nodes:,1
H2O cluster free memory:,2.400 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"locked, healthy"


## Descrição da Base de dados:

Problema de regressão. Traz informações do apartamento e informa a carga necessária para aquecimento ou refrigeração:

### Atributos:

* X1: Relative Compactness
* X2: Surface Area
* X3: Wall Area
* X4: Roof Area
* X5: Overall Height
* X6: Orientation # Rotation of the building shape (north, east)
* X7: Glazing area
* X8: Glazing area distribution # How windows are distributed
* Y1: Heating Load
* Y2: Cooling Load

In [6]:
data_path = "https://raw.githubusercontent.com/DarrenCook/h2o/bk/datasets/ENB2012_data.csv"
ENB_data = h2o.import_file(data_path)

ENB_data.describe()

factorsList = ["X6","X8"]
ENB_data[factorsList] = ENB_data[factorsList].asfactor()

train, test = ENB_data.split_frame([0.8])

x = ["X1", "X2", "X3", "X4", "X5", "X6", "X7", "X8"]
y = "Y2"  #Or "Y1"

Parse progress: |█████████████████████████████████████████████████████████| 100%
Rows:768
Cols:10




,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
type,real,real,real,real,real,int,real,int,real,real
mins,0.62,514.5,245.0,110.25,3.5,2.0,0.0,0.0,6.01,10.9
mean,0.7641666666666665,671.7083333333327,318.5000000000001,176.60416666666683,5.249999999999997,3.5000000000000018,0.234375,2.8124999999999982,22.307200520833337,24.587760416666658
maxs,0.98,808.5,416.5,220.5,7.0,5.0,0.4,5.0,43.1,48.03
sigma,0.10577747587475196,88.0861160558684,43.626481437994876,45.165950222920465,1.7511404367453502,1.1187625870603637,0.13322056291470774,1.55095966422475,10.09019573200616,9.513305562326165
zeros,0,0,0,0,0,0,48,48,0,0
missing,0,0,0,0,0,0,0,0,0,0
0,0.98,514.5,294.0,110.25,7.0,2.0,0.0,0.0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3.0,0.0,0.0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4.0,0.0,0.0,15.55,21.33


## Random Forest

Padrões:

max número de árvores = 50
max profundidade = 20 (controla a complexidade de cada árvore)

In [15]:
Random_forest = H2ORandomForestEstimator(model_id="RF_defaults", nfolds=10) # nfolds determina a validação cruzada
Random_forest.train(x, y, train)

Random_forest

drf Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  RF_defaults


ModelMetricsRegression: drf
** Reported on train data. **

MSE: 3.546518052665339
RMSE: 1.883220128573752
MAE: 1.3085243413398755
RMSLE: 0.058586591720126854
Mean Residual Deviance: 3.546518052665339

ModelMetricsRegression: drf
** Reported on cross-validation data. **

MSE: 3.3878256818480805
RMSE: 1.8406047054835213
MAE: 1.2899755589683335
RMSLE: 0.05733087348218399
Mean Residual Deviance: 3.3878256818480805
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid,cv_6_valid,cv_7_valid,cv_8_valid,cv_9_valid,cv_10_valid
mae,1.285542,0.0969215,1.2727948,1.3324128,1.3670131,1.347689,1.210411,1.5274715,1.0302113,1.3488773,1.3317478,1.0867914
mean_residual_deviance,3.3502119,0.4308272,3.0767028,4.007385,3.5522285,4.3660355,2.5989387,3.6668618,2.4661949,3.7736878,3.3561354,2.6379485
mse,3.3502119,0.4308272,3.0767028,4.007385,3.5522285,4.3660355,2.5989387,3.6668618,2.4661949,3.7736878,3.3561354,2.6379485
r2,0.9622551,0.0047765,0.9671108,0.9554455,0.9547865,0.9532502,0.9683892,0.9548921,0.9711002,0.9645776,0.9621419,0.9708575
residual_deviance,3.3502119,0.4308272,3.0767028,4.007385,3.5522285,4.3660355,2.5989387,3.6668618,2.4661949,3.7736878,3.3561354,2.6379485
rmse,1.822633,0.1187868,1.7540532,2.0018454,1.8847357,2.0895061,1.6121224,1.9149052,1.5704124,1.9425982,1.8319758,1.6241763
rmsle,0.0570639,0.0029194,0.0535044,0.0582909,0.0621287,0.0605292,0.0509648,0.0594544,0.0508903,0.0593079,0.0617546,0.0538140


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2018-09-23 15:27:13,3.170 sec,0.0,nan,nan,nan
,2018-09-23 15:27:13,3.175 sec,1.0,2.5146226,1.6118356,6.3233269
,2018-09-23 15:27:13,3.179 sec,2.0,2.6770853,1.6544800,7.1667856
,2018-09-23 15:27:13,3.184 sec,3.0,2.6113756,1.7020979,6.8192827
,2018-09-23 15:27:13,3.188 sec,4.0,2.4692221,1.6260974,6.0970576
---,---,---,---,---,---,---
,2018-09-23 15:27:13,3.386 sec,46.0,1.8958373,1.3192077,3.5941989
,2018-09-23 15:27:13,3.392 sec,47.0,1.8915504,1.3172934,3.5779631
,2018-09-23 15:27:13,3.396 sec,48.0,1.8899409,1.3179690,3.5718765
,2018-09-23 15:27:13,3.400 sec,49.0,1.8847511,1.3108511,3.5522868



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
X1,687282.25,1.0,0.2852939
X2,662301.5625000,0.9636529,0.2749243
X4,499485.2812500,0.7267542,0.2073386
X5,383248.3125000,0.5576287,0.1590881
X3,97898.6640625,0.1424432,0.0406382
X7,52679.9257812,0.0766496,0.0218677
X8,19437.8066406,0.0282821,0.0080687
X6,6698.3549805,0.0097461,0.0027805


### Random Forest - Grid search

#### Cartesian

In [16]:
import h2o.grid

Random_Forest_Grid = h2o.grid.H2OGridSearch( H2ORandomForestEstimator(nfolds=10),
  hyper_params = {
    "ntrees": [50, 100, 120],
    "max_depth": [40, 60],
    "min_rows": [1, 2]
    }
  )
Random_Forest_Grid.train(x, y, train)

drf Grid Build progress: |████████████████████████████████████████████████| 100%


In [17]:
Random_Forest_Grid

     max_depth min_rows ntrees  \
0           40      1.0    120   
1           60      1.0    100   
2           60      1.0     50   
3           40      2.0     50   
4           60      2.0    120   
5           40      2.0    120   
6           40      1.0    100   
7           60      1.0    120   
8           40      2.0    100   
9           40      1.0     50   
10          60      2.0     50   
11          60      2.0    100   

                                                       model_ids  \
0    Grid_DRF_py_7_sid_b396_model_python_1537715595091_3_model_8   
1    Grid_DRF_py_7_sid_b396_model_python_1537715595091_3_model_5   
2    Grid_DRF_py_7_sid_b396_model_python_1537715595091_3_model_1   
3    Grid_DRF_py_7_sid_b396_model_python_1537715595091_3_model_2   
4   Grid_DRF_py_7_sid_b396_model_python_1537715595091_3_model_11   
5   Grid_DRF_py_7_sid_b396_model_python_1537715595091_3_model_10   
6    Grid_DRF_py_7_sid_b396_model_python_1537715595091_3_model_4   
7    Grid_DRF

# Gradient Boosting Machines:


In [19]:
GBM = H2OGradientBoostingEstimator(model_id = "GBM_defaults", nfolds = 10)

GBM.train(x, y, train)

GBM

gbm Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_defaults


ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 1.1937368912050104
RMSE: 1.092582670192517
MAE: 0.7046061258811456
RMSLE: 0.03237817841407894
Mean Residual Deviance: 1.1937368912050104

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 2.255580697536307
RMSE: 1.50185908045206
MAE: 0.9468387266962067
RMSLE: 0.04414452547550878
Mean Residual Deviance: 2.255580697536307
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid,cv_6_valid,cv_7_valid,cv_8_valid,cv_9_valid,cv_10_valid
mae,0.9399843,0.1574018,0.5937156,0.7171463,1.1969008,0.5670162,1.0127196,1.0877723,0.9111239,1.1845775,1.0966073,1.0322639
mean_residual_deviance,2.20955,0.6901889,0.76778,1.2379324,3.33471,0.6902708,3.5886931,2.762417,1.8047614,2.6990082,2.6617754,2.5481515
mse,2.20955,0.6901889,0.76778,1.2379324,3.33471,0.6902708,3.5886931,2.762417,1.8047614,2.6990082,2.6617754,2.5481515
r2,0.975133,0.0073704,0.9901634,0.9862893,0.960759,0.9910001,0.9664707,0.9731414,0.9793822,0.9699494,0.9636006,0.9705742
residual_deviance,2.20955,0.6901889,0.76778,1.2379324,3.33471,0.6902708,3.5886931,2.762417,1.8047614,2.6990082,2.6617754,2.5481515
rmse,1.4416304,0.2561757,0.8762306,1.1126242,1.8261188,0.8308254,1.8943846,1.6620522,1.3434141,1.6428659,1.6314949,1.596293
rmsle,0.0431794,0.0054727,0.0366516,0.0349351,0.0508390,0.0265682,0.0491761,0.0495720,0.0402341,0.0484061,0.0478295,0.0475823


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2018-09-23 15:32:04,2.806 sec,0.0,9.5031075,8.5792825,90.3090530
,2018-09-23 15:32:04,2.812 sec,1.0,8.5904760,7.7320971,73.7962787
,2018-09-23 15:32:04,2.817 sec,2.0,7.7701614,6.9683161,60.3754084
,2018-09-23 15:32:04,2.825 sec,3.0,7.0378291,6.2813952,49.5310384
,2018-09-23 15:32:04,2.831 sec,4.0,6.3831148,5.6735693,40.7441545
---,---,---,---,---,---,---
,2018-09-23 15:32:05,3.061 sec,46.0,1.1459554,0.7424474,1.3132139
,2018-09-23 15:32:05,3.065 sec,47.0,1.1267350,0.7320643,1.2695318
,2018-09-23 15:32:05,3.071 sec,48.0,1.1215120,0.7274794,1.2577892
,2018-09-23 15:32:05,3.078 sec,49.0,1.0962537,0.7082173,1.2017722



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
X4,110554.9296875,1.0,0.3826473
X1,82415.1875000,0.7454682,0.2852514
X5,64251.2343750,0.5811702,0.2223832
X7,13620.0,0.1231967,0.0471409
X3,13408.8662109,0.1212869,0.0464101
X8,3680.0708008,0.0332873,0.0127373
X6,990.9569702,0.0089635,0.0034299
X2,0.0,0.0,0.0


# Generalized linear models

Engloba vários modelos lineares (regressão linear, regressão logística, etc).

_"Only one letter different from GBM, GLMs (generalized linear models) take a very
different approach. Whereas decision trees are based on logic, and deep learning is a
black box inspired by the human brain, GLMs are based on mathematics."_

Se baseia na distribuição dos dados.

Principais parâmetros:

* family: The probability distribuition of the response variable (regression: gaussian, poisson, gamma, tweedie, etc, and classification: multinomial or binimial)

* tweedie_link_power: Used when distribution is “tweedie.” Default is 1.

* tweedie_variance_power: Used when distribution is “tweedie.” Default is 0.
* link: Can be one of “family_default,” “identity,” “logit,” “log,” “inverse,” or “tweedie.” As the name suggests, “family_default” is the default, and is usually best.

## Regularização:

Continuando com os parâmetros:

* _alpha_: Uma forma de evitar overfiting (valor padrão = $0.5$)

**L1** = Lasso Regression: Transforma alguns coeficientes em zero (simplifica o problema)

**L2** = Ridge Regression: Não zera mas busca simplificar o problema tornando-os próximos de zero.

** Elastic Net:** Uma combinação dos dois modelos de regularização

O parâmetro _alpha_ controla a regularização utilizada:
_alpha_ = 1, Lasso
_alpha_ = 0, Ridge Regression.

Para combinar os dois (Elastic Net) defini-se o valor de _alpha_ entre 0 e  1.

* _lambda:_ O "quão forte" deve ser aplicada a regularização (padrão determinado por *lamda_search*)

**Dica:** Colocar *lambda_search = true*  

Isso faz com que o modelo seja treinado com diferentes valores de _lambda_ e o melhor resultado é escolhido

Outros parâmetros relacionados ao lambda são:
* *lambda_min_ratio:* Menor valor de _lambda_ utilizado no *lambda_search* (padrão é 0.0001)
* *nlambdas:* Número máximo de _lambdas_ a serem usados na busca

_" The default of –1 normally means it
will try 100 lambdas but when doing ridge regression, i.e., alpha=0.0 , it instead
defaults to 30 lambdas_"

* *max_active_predictors:* por padrão -1 (sem restrições) mas pode ser usado para que a busca acabe mais rápido.

Outros parâmetros definem como o problema será resolvido e existem ainda parâmetros que tratam de outros fatores de restrições como colinearidade e _missing values_ dentre outros.




In [21]:
glm = H2OGeneralizedLinearEstimator(model_id = "GLM_defaults", nfolds = 10)
glm.train(x, y, train)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [22]:
glm.summary()

GLM Model: summary



,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
,gaussian,identity,"Elastic Net (alpha = 0.5, lambda = 0.01703 )",16,13,1,py_7_sid_b396


Análise de importância das variáveis:

In [29]:
glm.varimp()
p_glm = glm.predict(test)

p_glm

glm prediction progress: |████████████████████████████████████████████████| 100%


predict
26.1322
25.7297
27.3891
27.1553
27.4842
10.4336
10.4336
10.7371
11.4684
12.8578


# Deep Learning Models:

Redes neurais com uma grande quantidade de camadas escondidas (deep)

## Principais parâmetros:

* número de epochs
* número de neurônios em cada camada
* número de camadas

*You’ll be able to see if more epochs is going to help or not by look‐
ing at the scoring history plot (on Flow, or plot it yourself with data
from h2o.scoreHistory(m) in R, or m.scoring_history() in
Python). If the line wobbles erratically, and the overall trend is
sideways, not down, then more epochs is unlikely to help; try one
of the other ideas. If it is merely getting rather flat, you have
entered the realm of diminishing returns but, if you don’t mind
waiting, then more epochs might give a small improvement*

## Funções de ativação:

Define a soma dos pesos em relação aos dados de entrada e saída. A escolha depende do problema que se está resolvendo

### Rectifier:

Função de ativação padrão, valores de saída variam entre 0 e 1. Geralmente costuma ser a opção mais rápida

### Tanh

Tangente hiperbólica, abrange outputs entre -1 e 1

### Maxout

Atribui os maiores valores

## Parâmetros:

* _hidden:_ número de camadas escondidas (200 por padrão, com duas camadas escondidas)
* auto-encoder: falso por padrão (aprendizado não superfisionado), se verdadeiro, recomenda-se usar  _tanh_ como função de ativação.

### Regularização:

* _activation:_ define as funções de ativação e o dropout (elimina conexões entre layers)

* *hidden_drouput_ratio* e *input_dropout_ratio:* definem as taxas de dropout das camadas escondidas e de entrada. Na cadama escondida, o padrão é 0.5, já na camada de entra é definido 0 por padrão (sem dropout)

* *l1* e *l2:* Lasso e ridge regression, 0.0 por padrão.

* *max_w2:* limite superior do quadrado da soma do quadrado dos pesos de entrada dos neurônios. Sem limite por padrão

### Deep Learning Scoring:

* *train_samples_per_iteration:* quantas amostras de treinamento serão usadas em cada iteração (quando o modelo será avaliado no final do ciclo). -2 por padrão (deixa o h2o decidir automaticamente)

* *score_interval:*  tempo mínimo entre a avaliação dos modelos (5 por padrão)

* *score_duty_cycle:*  Relação entre o tempo gasto em  _scoring_ e treinamento. (padrão é 0.1 - 10% scoring e 90% treinamento)

* *replicate_training_data:* verdadeiro por padrão. replica os dados em cada nó do cluster (H2O server). Para datasets menores isso pode tornar o treinamento mais rápido.

* *shuffle_training_data:* Falso por padrão. Caso verdadeiro, mistura os dados de forma aleatória





In [36]:
dl = H2ODeepLearningEstimator(model_id = "DL_defaults", nfolds = 10)
dl.train(x, y, train)

deeplearning Model Build progress: |██████████████████████████████████████| 100%
